In [3]:
from google.colab import files
uploaded = files.upload()

Saving homework.csv to homework.csv


In [1]:
!pip install tpot


In [2]:
!pip install pandas numpy scikit-learn matplotlib


In [4]:
import pandas as pd

print("1. Loading dataset...")
df = pd.read_csv("/content/homework.csv")

print("Dataset shape:", df.shape)
df.head()


1. Loading dataset...
Dataset shape: (546, 13)


,rownames,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
0,1,42000,5850,3,1,2,yes,no,yes,no,no,1,no
1,2,38500,4000,2,1,1,yes,no,no,no,no,0,no
2,3,49500,3060,3,1,1,yes,no,no,no,no,0,no
3,4,60500,6650,3,1,2,yes,yes,no,no,no,0,no
4,5,61000,6360,2,1,1,yes,no,no,no,no,0,no


In [11]:
target = "price"   # change if needed

X = df.drop(columns=[target, 'rownames']) # Drop 'rownames' and target column

# Identify categorical columns
categorical_cols = X.select_dtypes(include=['object']).columns

# Apply one-hot encoding to categorical columns
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

y = df[target]

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Training size:", X_train.shape)
print("Test size:", X_test.shape)


Training size: (436, 11)
Test size: (110, 11)


In [13]:
from tpot import TPOTRegressor

print("\n4. Running TPOT AutoML...")

tpot = TPOTRegressor(
    generations=5,
    population_size=20,
    random_state=42,
    n_jobs=1  # Changed from -1 to 1 to prevent worker timeout issues
)

tpot.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/tpot/tpot_estimator/estimator.py:458: UserWarning: Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.
  warnings.warn("Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.")
/usr/local/lib/python3.12/dist-packages/distributed/node.py:188: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 32835 instead
  warnings.warn(
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:41057
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:32835/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:33141'



4. Running TPOT AutoML...


INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:38085 name: 0
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:38085
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:48572
INFO:distributed.scheduler:Receive client connection: Client-d83c3b51-f0cd-11f0-8405-0242ac1c000c
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:48588

Generation:   0%|          | 0/5 [00:00<?, ?it/s]INFO:distributed.nanny:Closing Nanny gracefully at 'tcp://127.0.0.1:40041'. Reason: worker-close
INFO:distributed.core:Received 'close-stream' from tcp://127.0.0.1:44474; closing.
INFO:distributed.scheduler:Remove worker addr: tcp://127.0.0.1:42481 name: 0 (stimulus_id='handle-worker-cleanup-1768342878.634411')
ERROR:distributed.scheduler:Removing worker 'tcp://127.0.0.1:42481' caused the cluster to lose scattered data, which can't be recovered: {'DataFrame-04ec15d5648248ec897e0ca36937c6eb', 'Series-b066b96e4200dcd18b2e3cdad945d

KeyError: "None of [Index(['Eval Error'], dtype='object')] are in the [index]"

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

y_pred = tpot.predict(X_test)

print("R² Score:", r2_score(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))

In [ ]:
print("\nBest pipeline found by TPOT:")
print(tpot.fitted_pipeline_)

In [ ]:
tpot.export("tpot_best_model.py")